In [48]:
import pandas as pd
import numpy as np
import pickle

### Import data

In [49]:
nba = pd.read_csv('../data/gamelogs_advanced.csv', index_col=0)
nba.date_game = pd.to_datetime(nba.date_game,infer_datetime_format=True)

### Change win loss to numeric

In [51]:
nba['w_l'] = np.where(nba['game_result'] == 'W', 1, 0)

### Sort by season and team and create expanding mean columns

In [52]:
teams = nba.sort_values(['season', 'team'], ascending=[True, True])

In [53]:
unwanted_columns = ['team', 'season', 'game_season', 'date_game', 'game_location',
                    'opp_id', 'game_result']

columns_to_mean = [column for column in list(nba.columns) if column not in unwanted_columns]

In [55]:
def mean_expansion(df, column_list):
    for column in column_list:
        df[column+'_mean']=df.groupby(['season', 'team'])[column].transform(lambda x: x.expanding().mean().shift(1))
    return df

means = mean_expansion(teams, columns_to_mean)

In [57]:
filename = '../data/cleaned_stats.sav'
pickle.dump(means, open(filename, 'wb'))

### Make new dataframe consisting of only columns to be merged with schedule

In [58]:
cols = list(means.columns.values)
new_cols = [cols[0]]+cols[2:4]+[cols[7]]+cols[58:]
new_means = means[new_cols].copy()

## Create League Schedule

In [59]:
schedule = nba[['date_game', 'team', 'season', 'game_location', 
                'opp_id']].sort_values(by='date_game').reset_index(drop=True)

In [61]:
schedule['home_team'] = np.where(schedule['game_location'] == '@', schedule['opp_id'], schedule['team'])
schedule['away_team'] = np.where(schedule['game_location'] == '@', schedule['team'], schedule['opp_id'])

In [63]:
schedule.drop(['team', 'game_location', 'opp_id'], axis=1, inplace=True)

In [84]:
schedule.head()

,date_game,season,home_team,away_team
0,2007-10-30,2008,LAL,HOU
1,2007-10-30,2008,LAL,HOU
2,2007-10-30,2008,GSW,UTA
3,2007-10-30,2008,GSW,UTA
4,2007-10-30,2008,SAS,POR


In [66]:
schedule_new = schedule.drop_duplicates()

In [67]:
schedule_new.head()

,date_game,season,home_team,away_team
0,2007-10-30,2008,LAL,HOU
2,2007-10-30,2008,GSW,UTA
4,2007-10-30,2008,SAS,POR
6,2007-10-31,2008,ORL,MIL
7,2007-10-31,2008,MEM,SAS


In [68]:
filename = '../data/schedules.sav'
pickle.dump(schedule_new, open(filename, 'wb'))

### Merge dataframes for model construction

In [69]:
data = schedule_new.merge(new_means, left_on=['home_team', 'date_game'], right_on=['team', 'date_game'])

In [70]:
home_column_name_dict = {column:'home_'+column for column in new_cols}
data = data.rename(columns = home_column_name_dict)

In [73]:
data.head()

,home_date_game,season,home_team,away_team,home_team,home_game_season,home_pts,home_pts_mean,home_opp_pts_mean,home_fg_mean,...,home_blk_pct_mean,home_efg_pct_mean,home_tov_pct_mean,home_orb_pct_mean,home_ft_rate_mean,home_opp_efg_pct_mean,home_opp_tov_pct_mean,home_drb_pct_mean,home_opp_ft_rate_mean,home_w_l_mean
0,2007-10-30,2008,LAL,HOU,LAL,1,93,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-10-30,2008,GSW,UTA,GSW,1,96,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-10-30,2008,SAS,POR,SAS,1,106,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-10-31,2008,ORL,MIL,ORL,1,102,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-10-31,2008,MEM,SAS,MEM,1,101,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
data = data.merge(new_means, left_on=['away_team', 'home_date_game'], right_on=['team', 'date_game'])

In [76]:
away_column_name_dict = {column:'away_'+column for column in new_cols}
data=data.rename(columns = away_column_name_dict)

In [78]:
data=data.rename(columns = {'home_date_game':'date_game'})

In [79]:
data.head()

,date_game,season,home_team,away_team,home_team,home_game_season,home_pts,home_pts_mean,home_opp_pts_mean,home_fg_mean,...,away_blk_pct_mean,away_efg_pct_mean,away_tov_pct_mean,away_orb_pct_mean,away_ft_rate_mean,away_opp_efg_pct_mean,away_opp_tov_pct_mean,away_drb_pct_mean,away_opp_ft_rate_mean,away_w_l_mean
0,2007-10-30,2008,LAL,HOU,LAL,1,93,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-10-30,2008,GSW,UTA,GSW,1,96,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-10-30,2008,SAS,POR,SAS,1,106,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-10-31,2008,ORL,MIL,ORL,1,102,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-10-31,2008,MEM,SAS,MEM,1,101,NaN,NaN,NaN,...,6.2,0.506,7.5,27.3,0.207,0.538,15.8,77.8,0.167,1.0


In [80]:
data = data.loc[:,~data.columns.duplicated()]

In [81]:
data['score_margin']=data['home_pts']-data['away_pts']

In [82]:
data.head()

,date_game,season,home_team,away_team,home_game_season,home_pts,home_pts_mean,home_opp_pts_mean,home_fg_mean,home_fga_mean,...,away_efg_pct_mean,away_tov_pct_mean,away_orb_pct_mean,away_ft_rate_mean,away_opp_efg_pct_mean,away_opp_tov_pct_mean,away_drb_pct_mean,away_opp_ft_rate_mean,away_w_l_mean,score_margin
0,2007-10-30,2008,LAL,HOU,1,93,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2
1,2007-10-30,2008,GSW,UTA,1,96,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-21
2,2007-10-30,2008,SAS,POR,1,106,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
3,2007-10-31,2008,ORL,MIL,1,102,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
4,2007-10-31,2008,MEM,SAS,1,101,NaN,NaN,NaN,NaN,...,0.506,7.5,27.3,0.207,0.538,15.8,77.8,0.167,1.0,-3


In [83]:
filename = '../data/processed_data.sav'
pickle.dump(data, open(filename, 'wb'))